<a href="https://colab.research.google.com/github/dattali18/IR_Assignments/blob/main/Assignment.01/colab/01_IR_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Information Retrival Course

## Assigment 1

### Step 1: Data Cleaning

In [ ]:
# cleaning the data
!rm data.xlsx

rm: cannot remove 'data.xlsx': No such file or directory


In [ ]:
# downloding the data
import requests
import os
import pandas as pd

url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/data.xlsx?raw=true"
output_filename = "data.xlsx"

response = requests.get(url)
if response.status_code == 200:
    with open(output_filename, "wb") as file:
        file.write(response.content)

print(f"The file was downloaded, and it is in {output_filename}.")

The file was downloaded, and it is in data.xlsx.


In [ ]:
data_aj = pd.read_excel(output_filename, sheet_name="A-J", engine="openpyxl")
data_bbc = pd.read_excel(output_filename, sheet_name="BBC", engine="openpyxl")
data_jp = pd.read_excel(output_filename, sheet_name="J-P", engine="openpyxl")
data_nyt = pd.read_excel(output_filename, sheet_name="NY-T", engine="openpyxl")

In [ ]:
text_column_name = "Body Text"
title_column_name = "title"

In [ ]:
# crate a df from each sourcre with only the "Body Text" and "title" and rename the colmun into "text" and "title"
df_aj = data_aj[[text_column_name, title_column_name]].rename(columns={text_column_name: "text", title_column_name: "title"})

In [ ]:
# print the header of the df to see if sucessful

df_aj.head()

,text,title
0,pope francis has renewed calls for an immediat...,"pope renews call for gaza ceasefire, release o..."
1,united states president ronald reagan’s order ...,biden is still the best us president israel co...
2,the israeli military has launched more air str...,israeli air strikes continue across gaza as tr...
3,french police officers entered the paris insti...,police remove pro-palestinian students from pa...
4,NaN,mass graves found at southern gaza hospital ra...


In [ ]:
# do the same for all other

df_bbc = data_bbc[[text_column_name, title_column_name]].rename(columns={text_column_name: "text", title_column_name: "title"})
df_jp = data_jp[["Body", title_column_name]].rename(columns={"Body": "text", title_column_name: "title"})
df_nyt = data_nyt[[text_column_name, title_column_name]].rename(columns={text_column_name: "text", title_column_name: "title"})

Now we have 4 df with the column of text and title for each one of the 4 source and each data frame has about 600 articles.

In [ ]:
print(f"Article from A_J: {len(df_aj)}")
print(f"Article from BBC: {len(df_bbc)}")
print(f"Article from J_P: {len(df_jp)}")
print(f"Article from NYT: {len(df_nyt)}")

Article from A_J: 599
Article from BBC: 549
Article from J_P: 599
Article from NYT: 599


Now we will perform the data cleaning in two steps:

1. cleaning the word from all the punctuation marks.
  - for example "How are you?" -> "How are you ?" because "you" != "you?"
2. cleaning the documents with lemmatisation.
 - "cleaning" -> "clean"

 Outpute at the end:

 1. 4 cleaned document sets from the ponctuation (not deleting them).
 2. 4 cleaned document set by that include only the lemma (not word but their root).

In [ ]:
# performing the cleaning number 1
import re
import pandas as pd

def clean_text(text):
    # Normalize all types of single and double quotation marks to standard forms
    text = re.sub(r"[‘’`]", "'", text)  # Convert all single quote variations to '
    text = re.sub(r"[“”]", '"', text)   # Convert all double quote variations to "

    # Tokenize with regex to handle punctuation outside of words and contractions
    tokens = re.findall(r"\b\w+(?:'\w+)?\b|[^\w\s]", text, re.UNICODE)

    return ' '.join(tokens)


In [ ]:
# test the cleaning with a df
# we can perform more tests as we find extreme cases.

text = "How are you? I'm well thank you. doesn't."
clean_text(text)

"How are you ? I'm well thank you . doesn't ."

In [ ]:
# let's write the data cleaning


def clean_df_ponctuation(df):
  """
  This function should return a new df and not change the old one.
  """
  df_copy = df.copy()
  df_copy["text"] = df_copy["text"].astype(str).apply(clean_text)
  return df_copy

In [ ]:
# test the function with a data frame and print the
df_aj_ponctuation = clean_df_ponctuation(df_aj)

In [ ]:
# test the text that was produce

for i in range(5):
  print(f"{i + 1}: {data_aj_ponctuation['text'][i]}")

1: pope francis has renewed calls for an immediate ceasefire in gaza and the release of all israeli captives in a peace - focused address marking easter sunday , the most important day on the christian calendar .
2: united states president ronald reagan's order to israeli prime minister menachem begin to put an end to his " holocaust " in lebanon is perhaps the best - known political anecdote from israel's 1982 invasion .
3: the israeli military has launched more air strikes across the gaza strip to deadly effect as officials discuss the latest proposed deal to end hostilities and exchange prisoners .
4: french police officers entered the paris institute of political studies ( sciences po ) in paris and removed pro - palestinian student activists who had occupied its buildings to protest israel's war on gaza .
5: nan


In [ ]:
# clean all the other df

df_bbc_ponctuation = clean_df_ponctuation(df_bbc)
df_jp_ponctuation = clean_df_ponctuation(df_jp)
df_nyt_ponctuation = clean_df_ponctuation(df_nyt)

Now that we have clean the data in the first type we will do the lemmatisation.

In [ ]:
# installing the needed packeges
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 73.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy

# Load spaCy's English model
nlp = spacy.load("en_core_web_sm")

def lemmatize_text(text):
    # Normalize all types of single and double quotation marks to standard forms
    text = re.sub(r"[‘’`]", "'", text)  # Convert all single quote variations to '
    text = re.sub(r"[“”]", '"', text)   # Convert all double quote variations to "

    # Tokenize with regex to handle punctuation outside of words and contractions
    tokens = re.findall(r"\b\w+(?:'\w+)?\b|[^\w\s]", text, re.UNICODE)

    text = ' '.join(tokens)
    # Process the text with spaCy's NLP pipeline
    doc = nlp(text)
    # Extract and join lemmas for each token
    lemmatized_text = ' '.join(token.lemma_ for token in doc)
    return lemmatized_text


In [ ]:
# test the lemmatisation on simple and extreme cases

text = "How are you? I'm well thank you. doesn't. cleaning busiest"
lemmatize_text(text)

'how be you ? I be well thank you . do not . clean busy'

Ok now that we have a lemmatization function that works even on the extrem cases lets clean the text

In [ ]:
def clean_data_lemma(df):
  df_copy = df.copy()
  df_copy["text"] = df_copy["text"].astype(str).apply(lemmatize_text)
  return df_copy

In [ ]:
# test with df_aj and print the frist 5 text to test

data_aj_lemma = clean_data_lemma(df_aj)


In [ ]:
for i in range(5):
  print(f"{i + 1}:\n{data_aj_lemma['text'][i]}")
  print(f"{df_aj['text'][i]}\n\n")

1:
pope francis have renew call for an immediate ceasefire in gaza and the release of all israeli captive in a peace - focused address mark easter sunday , the most important day on the christian calendar .
pope francis has renewed calls for an immediate ceasefire in gaza and the release of all israeli captives in a peace-focused address marking easter sunday, the most important day on the christian calendar.


2:
united states president ronald reagan 's order to israeli prime minister menachem begin to put an end to his " holocaust " in lebanon be perhaps the well - know political anecdote from israel 's 1982 invasion .
united states president ronald reagan’s order to israeli prime minister menachem begin to put an end to his “holocaust” in lebanon is perhaps the best-known political anecdote from israel’s 1982 invasion.


3:
the israeli military have launch more air strike across the gaza strip to deadly effect as official discuss the late propose deal to end hostility and exchange p

In [ ]:
# now clean the rest of the df
df_aj_lemma = data_aj_lemma
df_bbc_lemma = clean_data_lemma(df_bbc)
df_jp_lemma = clean_data_lemma(df_jp)
df_nyt_lemma = clean_data_lemma(df_nyt)

In [ ]:
# store the all the cleaned data in files for further usage
# the name of the file should be <source>_word.csv with headr

df_aj_ponctuation.to_csv("A_J_word.csv", index=False)
df_bbc_ponctuation.to_csv("BBC_word.csv", index=False)
df_jp_ponctuation.to_csv("J_P_word.csv", index=False)
df_nyt_ponctuation.to_csv("NYT_word.csv", index=False)

In [ ]:
df_aj_lemma = data_aj_lemma

df_aj_lemma.to_csv("A_J_lemma.csv", index=False)
df_bbc_lemma.to_csv("BBC_lemma.csv", index=False)
df_jp_lemma.to_csv("J_P_lemma.csv", index=False)
df_nyt_lemma.to_csv("NYT_lemma.csv", index=False)

## Step 2: TF-TDF